<img src="images/header.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Grado</font></h2>

<h1><font color="#6B001F" size=5>Generación automática de playlist de canciones <br> mediante técnicas de minería de datos</font></h1>
<h2><font color="#92002A" size=3>Parte 4 - Filtrado de playlists</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Grado en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---
<br>

<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>

* [1. Introducción](#section1)
* [2. Carga de resultados](#section2)
* [3. Comprobación de valores perdidos](#section3)
* [4. Filtrado de playlists](#section4)
    * [4.1. Filtrado por número de pistas, artistas y álbumes](#section41)
    * [4.2. Filtrado de playlists con idénticas características](#section42)
    * [4.3. Filtrado por número de ediciones](#section43)
    * [4.4. Filtrado por títulos](#section44)
        * [Número de caracteres](#section441)
        * [Número de palabras](#section442)
        * [Existencia de emoticonos](#section443)
        * [Alfabeto](#section444)
        * [Idioma](#section445)
        * [Títulos ofensivos](#section446)
    * [4.5. Consideraciones en el filtrado de playlists](#section45)
* [5. Almacenamiento de resultados](#section5)

<br>

---

In [1]:
# Permite establecer la anchura de la celda
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import csv
import emoji
import json
import os
import pandas as pd
import requests
import urllib.parse
import zipfile

from collections import defaultdict
from emoji import UNICODE_EMOJI
from shutil import copyfile
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_nb

---

<br>

<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>
<br>

Una vez que ya hemos descargado aquellas listas de reproducción que nos interesaba estudiar con más detalle, vamos a realizar el proceso completo de filtrado. Nuestro objetivo es quedarnos con _1.000.000 de playlists_. También aprovecharemos este filtrado para obtener un conjunto de prueba.

Recordemos aquellos criterios que ya hemos aplicado en anteriores libretas para descartar listas de reproducción:

1. Eliminación de playlists con identificadores repetidos (se ha obtenido la misma lista tras buscar con diferentes términos).<sup>1</sup>
2. Tienen menos de 5 pistas o más de 250 pistas.<sup>1</sup>
3. El título contiene menos de 2 caracteres o más de 50 caracteres (sin contar espacios).<sup>1</sup>
4. El título contiene más de 9 palabras.<sup>1</sup>
5. No tienen ningún seguidor.<sup>2</sup>
6. Están marcadas como privadas.<sup>2</sup>
7. Contienen pistas locales.<sup>2</sup>

En esta libreta también volveremos a aplicar los criterios 2, 3 y 4, puesto que durante el proceso de búsqueda y el proceso de descarga han transcurrido varios días y el usuario podría haber modificado el título o las pistas que pertenecen a la playlist.

Por último, en caso de que no hayamos conseguido reducir el número de playlists, añadiremos dos últimos criterios:
- Playlists que contengan artistas poco frecuentes (sólo aparecen en 1 ó 2 listas).
- Número de seguidores inferior a 2.


<br>

<sup>1</sup>
<font size="2"> _Ver libreta_ ['Preparación de datos para el proceso de descarga'](02-PreparacionDescarga.ipynb).</font>

<sup>2</sup>
<font size="2"> _Ver libreta_ ['Descarga de playlists'](03-DescargaPlaylists.ipynb).</font>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section2"></a>
## <font color="#92002A">2 - Carga de resultados</font>

<br>

Antes de comenzar con el estudio de las playlist obtenidas en el proceso de descarga, definimos aquellos directorios con los que vamos a trabajar:

In [3]:
# Variables globales

# Directorio donde se encuentran almacenados los conjuntos 
# de playlists que hemos descargado
PLS_SETS_PATH = 'pls_sets'

# Directorio empleado para guardar/leer aquellos datos generados
# que resultan de interés para diferentes tareas
DATA_PATH = 'data'

# Directorio que empleamos para guardar copias de seguridad
BACKUP_PATH = 'backup'

<br>

Para obtener la información que nos faltaba por comprobar en el proceso de limpieza de playlist, creamos la función ***get_playlist_info***. Esta función se encargara de obtener el número de artistas y álbumes diferentes que hay en una playlist junto a la duración total de ésta.

In [4]:
# Genera la información adicional de una playlist que nos resultará
# útil para su filtrado
def get_playlist_info(track_list):
    """
    :param track_list: Lista de tracks a analizar.
    :return: Información generada. Tupla (artistas,álbumes,duración).
    """
    albums = set()
    artists = set()
    duration = 0
    for track in track_list:
        albums.add(track['album_uri'].split(':')[-1])
        artists.add(track['artist_uri'].split(':')[-1])
        duration += track['duration_ms']
    return len(artists), len(albums), duration, list(artists)

<br>

El siguiente método nos devuelve el número de veces que se ha editado una playlist, mediante las fechas obtenidas de la variable *added_at* de cada pista.

<br>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__:
Se considera que las pistas agregadas en una ventana de dos horas han sido agregadas en una sola sesión de edición.
</div>

<br>

In [5]:
def get_playlist_edits(date_list):
    """
    :param date_list: Lista de fechas a estudiar.
    :return: Número de veces que se ha editado.
    """
    if len(date_list) < 1:
        return None
    else:
        dates = date_list.copy()
        dates.sort()
        edits_count= 1
        n_init = dates.pop(0)
        while len(dates) > 0:
            num = dates.pop(0)
            # 2h -> 7200s
            if num > n_init + 7200:
                n_init = num
                edits_count += 1
        return edits_count

<br>

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: En nuestro caso, vamos a contar también como una edición la primera vez que se añadieron pistas a la playlist. Por lo que tendríamos lo siguiente:
- Si el número de ediciones es 1, la playlist no se ha modificado tras su creación.
- Si el número de ediciones es igual o superior a 2, la playlist se ha editado al menos una vez tras su creación.
</div>

<br>

El método ***get_playlist_date_info*** nos devuelve la información procedente del estudio de las fechas en las que fueron añadidas las pistas a la playlist:

- La última fecha en la que la lista ha sido modificada (*modified_at*).
- El número de veces que ha sido editada (*num_edits*).


In [6]:
def get_playlist_date_info(track_list):
    """
    :param date_list: Lista de pistas a estudiar.
    :return: Tupla con el número de veces que se ha editado y la última fecha de modificación.
    """
    if len(track_list) < 1:
        return None, None
    else:
        dates = [track['added_at'] for track in track_list]
        modified_at = max(dates)
        num_edits = get_playlist_edits(dates)

        return modified_at, num_edits

<br>

Creamos un DataFrame donde almacenaremos todas las playlists, junto a la información que queremos estudiar. Almacenaremos el resultado de este diccionario en un archivo `.csv` comprimido para tener una copia de seguridad y, como la duración del proceso es considerable, evitar volver a crear el diccionario desde 0 en caso de querer ejecutar la libreta de nuevo.

<br>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__:
Para evitar que el DataFrame resultante de leer todas las playlist descargadas tuviera un tamaño demasiado grande, y ya que para el filtrado que vamos a realizar no nos es necesario, se ha eliminado la lista de pistas que pertenece a una lista de reproducción. Por si nos fuese de necesidad recuperar dichos datos, añadimos el nombre del archivo `.zip` donde se encuentra la playlist.
</div>

<br>

Mientras se realiza este proceso, también vamos a crear un archivo en el que almacenaremos qué artistas pertenecen a una playlist para posteriormente aplicar dicho filtrado (en caso de ser necesario).

<br>

In [7]:
def get_downloaded_playlists_df(pl_sets_path):
    # La información obtenida será almacenada en un diccionario
    # que utilizaremos para crear el DataFrame por primera vez
    pls_dict = {}
    
    csv_file_path = os.path.join(DATA_PATH,'downloaded_pls_artists.csv')
    
    # Para cada fichero del directorio, lo descomprimimos y lo leemos
    # para obtener la información que nos es relevante para el filtrado
    for file_name in tqdm_nb(os.listdir(pl_sets_path)):
        if file_name.startswith('pls-set') and file_name.endswith('.zip'):
            file_path = os.path.join(pl_sets_path, file_name)
            with zipfile.ZipFile(file_path,'r') as zip_file:
                with zip_file.open(zip_file.namelist()[0]) as json_file:
                    data = json_file.read() 
            data = json.loads(data.decode())
            
            artists_dict = dict()
            
            for d in data:      
                num_artists, num_albums, duration, artists_list = get_playlist_info(d['tracks'])
                d['num_artists'] = num_artists
                d['num_albums'] = num_albums
                d['duration_ms'] = duration
                artists_dict[d['id']] = "|".join(artists_list)

                modified_at, num_edits = get_playlist_date_info(d['tracks'])
                d['modified_at'] = modified_at
                d['num_edits'] = num_edits

                del d['tracks']

                pl_id = d['id']
                del d['id']
                
                d['file_name'] = file_name

                pls_dict[pl_id] = d
                
            with open(csv_file_path, 'a+', newline='') as artists_file:
                csv_writer = csv.writer(artists_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
                for key,value in artists_dict.items():
                    csv_writer.writerow([key, value])
    
    # Creamos el DataFrame a partir del diccionario generado
    df_downloaded_pls = pd.DataFrame.from_dict(pls_dict, orient='index')
    df_downloaded_pls.index.name = 'id'
    
    return df_downloaded_pls

In [8]:
# Nombre del DataSet comprimido donde se encuentra almacenada la 
# información de las playlists tras generarse el DataFrame
# por primera vez
playlists_file = os.path.join(DATA_PATH,'downloaded_pls.csv')
playlists_artists_file = os.path.join(DATA_PATH,'downloaded_pls_artists.csv')

# En caso de que el fichero exista, lo cargamos para evitar 
# repetir el proceso de creación
if os.path.isfile(playlists_file) and os.path.isfile(playlists_artists_file):
    df_playlists_info = pd.read_csv(playlists_file, sep=';',
                                    encoding='utf-8', index_col=0)
else:
    # Si no existe la carpeta 'DATA_PATH', la creamos
    if not os.path.exists(DATA_PATH):
        os.makedirs(DATA_PATH)
    df_playlists_info = get_downloaded_playlists_df(PLS_SETS_PATH)
    
    # Volcamos el contenido del DataFrame a un fichero .csv
    csv_file_path = os.path.join(DATA_PATH,'downloaded_pls.csv')
    df_playlists_info.to_csv(csv_file_path, sep=';', encoding='utf-8')
    
    # Hacemos una copia de seguridad del archivo .csv
    copyfile(csv_file_path, os.path.join(BACKUP_PATH, 'downloaded_pls.csv'))

print(f"Número de playlists obtenidas: {len(df_playlists_info)}")

Número de playlists obtenidas: 3122640


<br>

Para comprobar que el DataFrame ha sido creado de forma correcta, mostramos las primeras 5 filas:

In [9]:
df_playlists_info.head()

,collaborative,name,num_tracks,num_followers,num_artists,num_albums,duration_ms,modified_at,num_edits,file_name
id,,,,,,,,,,
3FI0aZZz9hUw5qK0K2GDwH,False,Love Letter,17,65.0,16,16,3601989,1.556283e+09,5.0,pls-set_00000.zip
381M0rlWt8mB0CayapCcRr,False,skin deep,35,1.0,32,35,7146193,1.558387e+09,16.0,pls-set_00000.zip
3nSWAjR9QuXhb3YqO24HHw,False,junior year,250,1.0,113,184,57370653,1.556406e+09,113.0,pls-set_00000.zip
5M6FDo7oQ33CvNMaMiZ1uH,False,BOOM BOOM,20,1.0,16,20,4120665,1.555591e+09,2.0,pls-set_00000.zip
43QPMRPsHlbXILXIhYoxF6,False,Born in the Wrong Era,168,1.0,112,146,36941201,1.559002e+09,86.0,pls-set_00000.zip


<br>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section3"></a>
## <font color="#92002A">3 - Comprobación de valores perdidos</font>
<br>

Antes de comenzar a filtrar los resultados, vamos a comprobar que no hay valores perdidos (**NaN**):

In [10]:
df_playlists_info.isna().sum()

collaborative      0
name               7
num_tracks         0
num_followers    255
num_artists        0
num_albums         0
duration_ms        0
modified_at      724
num_edits        724
file_name          0
dtype: int64

Vemos que existen valores perdidos en las siguientes variables:

* *num_followers*
* *modified_at*
* *num_edits*

En el caso del número de seguidores, vamos a considerar que un valor **NaN** equivale a que la lista de reproducción no tiene ningún seguidor (*num_followers = 0*). 

Para las dos variables restantes: si no existe fecha de última modificación ni número de veces que se ha editado una playlist, la descartaremos por no poder calcular dichos valores.

Por lo tanto, podemos descartar aquellas filas que contengan un valor perdido:

In [11]:
df_playlists_info.dropna(inplace=True)

Para asegurarnos de que se han eliminado correctamente, vamos a volver a comprobar los valores perdidos del DataFrame `df_playlists_info`:

In [12]:
df_playlists_info.isna().sum() 

collaborative    0
name             0
num_tracks       0
num_followers    0
num_artists      0
num_albums       0
duration_ms      0
modified_at      0
num_edits        0
file_name        0
dtype: int64

Como podemos apreciar, nuestro DataFrame ya no posee ningún valor **NaN**.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section4"></a>
## <font color="#92002A">4 - Filtrado de playlists</font>
<br>

Una vez que hemos recopilado toda la información que nos va a hacer falta para filtrar las playlists y hemos eliminado los valores perdidos del DataFrame, comenzamos el proceso de filtrado.

<br>

<a id="section41"></a>
### <font color="#B20033">4.1 - Filtrado por número de pistas, artistas y álbumes</font>
<br>

Deben de cumplirse las siguientes condiciones:
1. El número de pistas de una playlist estará comprendido entre 5 y 250.
2. La playlist tendrá, al menos, 3 artistas diferentes.
3. La playlist tendrá, al menos, 2 álbumes diferentes.



In [13]:
df_playlists_info.drop(df_playlists_info[df_playlists_info['num_tracks'] > 250].index,inplace=True)
df_playlists_info.drop(df_playlists_info[df_playlists_info['num_tracks'] < 5].index,inplace=True)
print(f"Tamaño df_playlists_info (con filtrado de pistas): {len(df_playlists_info)}")

Tamaño df_playlists_info (con filtrado de pistas): 3114589


In [14]:
df_playlists_info.drop(df_playlists_info[df_playlists_info['num_artists'] < 3].index,inplace=True)
print(f"Tamaño df_playlists_info (con filtrado de artistas): {len(df_playlists_info)}")

Tamaño df_playlists_info (con filtrado de artistas): 2954967


In [15]:
df_playlists_info.drop(df_playlists_info[df_playlists_info['num_albums'] < 2].index,inplace=True)
print(f"Tamaño df_playlists_info (con filtrado de álbumes): {len(df_playlists_info)}")

Tamaño df_playlists_info (con filtrado de álbumes): 2954951


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section42"></a>
### <font color="#B20033">4.2 - Filtrado de playlists con idénticas características</font>
<br>

Hay casos en los que existen listas de reproducción iguales, pero con diferente título y pertenecientes a usuarios distintos. Para dichos casos, vamos a quedarnos con el primer resultado y borrar el resto:


In [16]:
df_playlists_info.drop_duplicates(subset=['duration_ms','num_tracks',
                                          'num_artists','num_albums'], 
                                  inplace=True, keep='first')
print(f"Tamaño df_playlists_info (con filtrado de duplicados): {len(df_playlists_info)}")

Tamaño df_playlists_info (con filtrado de duplicados): 2951179


<br>

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Que distintas playlists tengan valores idénticos para las siguientes variables:
- Duración de la playlist.
- Número de canciones.
- Número de artistas diferentes.
- Número de álbumes diferentes.

implicaría, con una probabilidad bastante alta, de que sea la misma playlist pero con título diferente. Aunque pueden darse casos extremos en los que, coincidiendo esos datos, pueda tratarse de otra lista de reproducción distinta. En nuestro caso vamos a considerar que cuando coincidan dichos valores, se trata de la misma playlist.
</div>


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section43"></a>
### <font color="#B20033">4.3 - Filtrado por número de ediciones</font>
<br>

Para nuestro conjunto de playlists, vamos a quedarnos con aquellas que han sido editadas al menos 1 vez tras su creación:

<br>


In [17]:
df_playlists_info.drop(df_playlists_info[df_playlists_info['num_edits'] < 2].index,inplace=True)
print(f"Tamaño df_playlists_info (con filtrado por número de ediciones): {len(df_playlists_info)}")

Tamaño df_playlists_info (con filtrado por número de ediciones): 2768027


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section44"></a>
### <font color="#B20033">4.4 - Filtrado por títulos</font>
<br>

Al igual que hicimos en la libreta [_Preparación de datos para el proceso de descarga_](02-PreparacionDescarga.ipynb), vamos a quedarnos con aquellas listas de reproducción cuyos títulos cumplan las siguientes condiciones:

- Tiene entre 5 y 50 caracteres (eliminando espacios en blanco).
- Tienen menos de 10 palabras.
- En caso de que el título contenga emoticonos:
    - Si el título también contiene texto, el número máximo de emoticonos es 10.
    - Sí el título sólo contiene emoticonos, el número máximo será de 4.
- Los caracteres del título pertenecen al alfabeto latino, al conjunto de caracteres comunes o son emoticonos.
- Filtrado de títulos por idioma.
- Filtrado de títulos ofensivos.


<a id="section441"></a>
#### <font color="#B20033">Número de caracteres</font> <br>

In [18]:
large_names_ids = df_playlists_info[(df_playlists_info.name.str.len()-df_playlists_info.name.str.count(' ')) > 50].index
sort_names_ids = df_playlists_info[(df_playlists_info.name.str.len()-df_playlists_info.name.str.count(' ')) < 2].index
df_playlists_info.drop(large_names_ids,inplace=True)
df_playlists_info.drop(sort_names_ids,inplace=True)
print(f"Tamaño df_playlists_info (con filtrado de longitud de títulos): {len(df_playlists_info)}")

Tamaño df_playlists_info (con filtrado de longitud de títulos): 2767447


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section442"></a>
#### <font color="#B20033">Número de palabras</font> <br>

In [19]:
large_names_ids = df_playlists_info[df_playlists_info.name.str.count(' ') > 10].index
df_playlists_info.drop(large_names_ids,inplace=True)
print(f"Tamaño df_playlists_info (con filtrado por número de palabras en título): {len(df_playlists_info)}")

Tamaño df_playlists_info (con filtrado por número de palabras en título): 2751987


<br>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__:
Los siguientes criterios que vamos a aplicar para filtrar las playlists mediante los títulos, podrían haberse llevado a cabo en el proceso de preparación de descarga (ver libreta [*Preparación para la descarga de playlists*](02-PreparacionDescarga.ipynb)). Para evitar tanta duplicidad a la hora de realizar el filtrado de playlists, y puesto que no aportaba mucha mejora a la hora de reducir el número de listas a descargar, se decidió no aplicar estos criterios en dicho proceso.
</div>

<br>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>


---

<a id="section443"></a>
#### <font color="#B20033">Existencia de emoticonos</font> <br>

In [20]:
# Comprueba si el carácter es un emoticono
def is_emoji(s):
    return s in UNICODE_EMOJI

# Comprueba si un texto contiene emoticonos
def contains_emoji(text):
    return len([c for c in UNICODE_EMOJI if(c in str(text))]) > 0

# Comprueba si un texto contiene sólo emoticonos
def all_text_emoji(text):
    return all([(c in UNICODE_EMOJI) for c in str(text)])

# Cuenta los emoticonos que contiene un texto
def count_text_emoji(text):
    return sum([(c in UNICODE_EMOJI) for c in str(text)])

# Elimina los emoticonos de un texto
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', str(text))

In [21]:
# Indica si el texto con emoticonos es válido según lo establecido
def invalid_name_emojis(name):
    if (all_text_emoji(name) and len(name) > 4):
        return True
    else:
        return count_text_emoji(name) > 10

In [22]:
df_playlists_info.drop(df_playlists_info[df_playlists_info['name'].apply(invalid_name_emojis)].index, inplace=True)
print(f"Tamaño df_playlists_info (con filtrado de títulos con emoticonos): {len(df_playlists_info)}")

Tamaño df_playlists_info (con filtrado de títulos con emoticonos): 2751525


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---


<a id="section444"></a>
#### <font color="#B20033">Alfabeto</font> <br>

Cuando estudiamos el [_Million Playlist Dataset_](00-EstudioMPD.ipynb), vimos que los caracteres pertenecían al alfabeto latino, al conjunto de caracteres comunes y/o eran emoticonos. En dicho caso, lo comprobamos mediante expresiones regulares.

En esta ocasión, para acotar más el número de playlists y controlar mejor aquellos caracteres con los que vamos a trabajar,  vamos a limitar el alfabeto a los siguientes caracteres:

```
' .·,:@#$€%&~°|)(><}{][+-*/%?¿!¡_–—0123456789abcdefghijklmnopqrstuvwxyzçñáéíóúàèìòùäëïöüâêîôû
``` 

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Hemos ignorado los caracteres `\` y `;` por los siguientes motivos: 
- La barra invertida, `\`, la ignoramos para evitar las secuencias de escape (tabulador, salto de línea, etc...).
- El punto y coma, `;`, lo ignoramos ya que lo vamos a usar para separar los campos en los archivos `.csv`
</div> <br>

In [23]:
# Identifica aquellos títulos que no contienen letras fuera
# del diccionario que hemos establecido.
def contains_invalid_chars(text):
    text = str(text).lower()
    return any((c not in "' .·,:@#$€%&~°|)(><}{][+-*/%?¿!¡_–—0123456789abcdefghijklmnopqrstuvwxyzçñáéíóúàèìòùäëïöüâêîôû" and not is_emoji(c)) for c in text)

In [24]:
df_playlists_info.drop(df_playlists_info[df_playlists_info['name'].apply(contains_invalid_chars)].index, inplace=True)
print(f"Tamaño df_playlists_info (con filtrado de caracteres no válidos): {len(df_playlists_info)}")

Tamaño df_playlists_info (con filtrado de caracteres no válidos): 2654225


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---


<a id="section445"></a>
#### <font color="#B20033">Idioma</font> <br>

Otro de los criterios que se aplicaron al crear el _Million Playlist Dataset_,  fue que las listas de reproducción pertenecían a usuarios de los Estados Unidos de América. Como la WebAPI de _Spotify_ no nos proporciona esta información, vamos a identificar el idioma de los títulos de las playlists y nos quedaremos con aquellos que estén en inglés o que contengan únicamente emoticonos y/o otros caracteres como números, signos de puntuación, etc...

Para clasificar los títulos de las playlists por su idioma, se ha empleado el servicio [_Text Analytics_](https://azure.microsoft.com/services/cognitive-services/text-analytics/) de [**Azure Cognitive Services**](https://azure.microsoft.com/es-es/services/cognitive-services/).

<br>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__:
Para ver cómo funciona la detección del idioma mediante *Text Analytics*, se puede consultar un ejemplo publicado en [_Microsoft Docs_](https://docs.microsoft.com): 
- [Example: How to detect language with Text Analytics](https://docs.microsoft.com/azure/cognitive-services/text-analytics/how-tos/text-analytics-how-to-language-detection)
</div> <br>

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Para hacer uso del servicio, se necesita una clave de acceso. Dicha clave puede obtenerse siguiendo los pasos que se indican en el siguiente enlace:
- [Get an access key for the Text Analytics API](https://docs.microsoft.com/azure/cognitive-services/text-analytics/how-tos/text-analytics-how-to-access-key)
</div> <br>

<div class="alert alert-danger">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Advertencia__:
Este servicio de _Microsoft Azure_ es de **pago**<sup>1</sup>, pero se puede hacer uso de él mediante la instancia _'Gratis - Web/Container'_ que permite realizar 5.000 transacciones al mes de forma gratuita.

<font size=1><sup>1</sup> Se recomienda consultar la [información de precios del servicio](https://azure.microsoft.com/es-es/pricing/details/cognitive-services/text-analytics/).<font>

</div>

<br>

Mediante el método ***get_names_language***, obtenemos el idioma de cada uno de los títulos de las playlist que tenemos en nuestro DataFrame. Posteriormente los guardaremos en un fichero `.json` comprimido. Guardamos una copia de dicha información por los siguientes motivos:
- El proceso puede durar más de 1 hora.
- Si volvemos a realizar las llamadas al servicio, se nos vuelve a facturar por el uso que hacemos (en caso de usar alguna de las instancias de pago). En caso de hacer uso de la instancia gratuita, podríamos agotar innecesariamente las 5.000 instancias al mes que nos facilitan.

Para realizar la llamada al servicio, necesitamos convertir nuestros datos al siguiente formato *JSON*: 

<br>

```
{ 
    "documents" : 
    [ 
        {"id" : "id_0", "text" : "title_0"},
        
        ...
        
        {"id" : "id_n", "text" : "title_n"}
    ] 
}
```
<br>

Como podemos apreciar en el esquema anterior, enviamos un diccionario de un único elemento con clave '*documents*' cuyo valor es una lista de diccionarios, los textos de los cuales deseamos identificar su idioma. En una llamada al servicio podemos adjuntar hasta 1.000 textos, almacenados cada uno en un diccionario de dos elementos, dentro del fichero *JSON*. Estos elementos son:
- ___id___ : Identificador que damos al texto.
- ___text___ : Texto del que deseamos obtener el idioma.

A continuación, mostramos el ejemplo de un JSON empleado para la llamada al servicio:

<br>

```
{
     "documents": [
         {
             "id": "1",
             "text": "This document is in English."
         },
         {
             "id": "2",
             "text": "Este documento está en inglés."
         },
         {
             "id": "3",
             "text": "Ce document est en anglais."
         },
         {
             "id": "4",
             "text": "本文件为英文"
         },                
         {
             "id": "5",
             "text": "Этот документ на английском языке."
         }
     ]
 }
```

<br>

In [25]:
def get_names_language(df_info):
    # Lista de diccionarios que emplearemos para almacenar
    # el identificador y el nombre de las playlists
    items = []
    
    # Forma más rápida de obtener el id y el título de cada
    # playlist en vez de iterar sobre el DataFrame
    index_name_tuplist = list(zip(df_info.index.to_list(), 
                                  df_info.name.to_list()))
    
    # Para cada elemento de la tupla, lo convertimos a
    # un diccionario y lo añadimos a la lista 'items'
    for tup in index_name_tuplist:
        items.append({'id' : tup[0], 'text': tup[1]})

    # Creamos bloques (chunks) de 1.000 elementos, ya que
    # podemos hacer peticiones al servicio que contengan
    # hasta 1.000 textos (1 texto = 1 transacción)
    chunk_len = 1000
    chunks_data = [items[x:x+chunk_len] for x in range(0, len(items), chunk_len)]
    
    # Lista que emplearemos para guardar las respuestas que
    # nos devuelve el servicio
    chunks_response = []
    
    access_key = '' # API Key de Azure Cognitive Services
    end_point = '' # Endpoint Azure Cognitive Services
    request_url = urllib.parse.urljoin(end_point, 'text/analytics/v2.1/languages')
    
    # Cabecera requerida para hacer la petición al servicio
    headers = {'Ocp-Apim-Subscription-Key' : access_key,
               'Content-Type' : 'application/json',
               'Accept' : 'application/json'}

    # Para cada bloque de 1.000 items, realizamos una 
    # llamada al servicio y almacenamos su respuesta
    with tqdm(total=len(chunks_data)) as pbar:
        for i , chunk in enumerate(chunks_data):
            json_data = {'documents': chunk}
            
            res = requests.post(request_url, headers=headers, json=json_data)
            
            # Comprobamos que la ejecución es correcta. En caso
            # contrario, imprimimos un mensaje con el bloque
            # donde se ha producido el error
            if res.status_code == 200:
                chunks_response.append(res.json()['documents'])
            else:
                print(f'Error al obtener chunk[{i}]')
            
            pbar.update(1)
    
    # Almacenamos en una única lista el resultado de la
    # identificación del idioma de cada título que se 
    # encuentra repartido en los distintos bloques
    language_data = []
    
    for chunk in chunks_response:
        for item in chunk:
            language_data.append(item)
    
    # Volcamos la información a un archivo .json
    lang_file_name = 'lang_results_list.json'
    lang_file_path = os.path.join(BACKUP_PATH, lang_file_name)
    with open(lang_file_path, 'w') as file:
        json.dump(language_data,file,indent=4)
        
    # Comprimimos el fichero .json y eliminamos el fichero .json
    lang_zip_path = os.path.join(BACKUP_PATH,'lang_results_list.zip')
    with zipfile.ZipFile(lang_zip_path,'w') as zip_file: 
         zip_file.write(lang_file_path, compress_type=zipfile.ZIP_DEFLATED, arcname=lang_file_name)
        
    os.remove(lang_file_path)
    
    return language_data

<br>

El formato de la respuesta que obtendremos tras la llamada al servicio _language_ de _Text Analytics_ es la siguiente:

<br>


```
{ 
    "documents" : 
    [ 
        {"id" : "id_0",
         "detectedLanguages": 
         [
             {
                 "name": "language_0",
                 "iso6391Name": "l_0",
                 "score": s_0
            }
        ],
        
        ...
        
        {"id" : "id_n",
         "detectedLanguages": 
         [
             {
                 "name": "language_n",
                 "iso6391Name": "l_n",
                 "score": s_n
            }
        ]
    ] 
}
```

<br>

La respuesta es similar a la llamada, salvo que en este caso el segundo valor del diccionario es otra lista de diccionarios, llamada '*detectedLanguages*' , con el idioma que ha identificado (en dos formatos) y su puntuación. Los elementos que componen la identificación al idioma del texto, con el _id_ que hemos establecido, son los siguientes:
- ***name*** : Nombre del idioma identificado.
- ***iso6391Name*** : Nombre del idioma identificado en formato [_ISO 639-1_](https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes).
- ***score*** : Puntuación obtenida por la identificación.



A continuación, mostramos la respuesta que se obtendría tras la identificación de los textos del ejemplo anterior:

<br>

```
{
    "documents": [
        {
            "id": "1",
            "detectedLanguages": [
                {
                    "name": "English",
                    "iso6391Name": "en",
                    "score": 1
                }
            ]
        },
        {
            "id": "2",
            "detectedLanguages": [
                {
                    "name": "Spanish",
                    "iso6391Name": "es",
                    "score": 1
                }
            ]
        },
        {
            "id": "3",
            "detectedLanguages": [
                {
                    "name": "French",
                    "iso6391Name": "fr",
                    "score": 1
                }
            ]
        },
        {
            "id": "4",
            "detectedLanguages": [
                {
                    "name": "Chinese_Simplified",
                    "iso6391Name": "zh_chs",
                    "score": 1
                }
            ]
        },
        {
            "id": "5",
            "detectedLanguages": [
                {
                    "name": "Russian",
                    "iso6391Name": "ru",
                    "score": 1
                }
            ]
        }
    ]
}
```

<br>

Realizamos el proceso de obtención del idioma de los títulos. En caso de existir un fichero comprimido con el nombre `lang_results_list.zip` lo cargamos, puesto que contiene la información que necesitamos:

In [26]:
lang_zip_path = os.path.join(BACKUP_PATH,'lang_results_list.zip')

if os.path.isfile(lang_zip_path):
    with zipfile.ZipFile(lang_zip_path,'r') as zip_file:
        with zip_file.open(zip_file.namelist()[0]) as zip_file:
            data = zip_file.read()
            language_data = json.loads(data.decode())
else:
    language_data = get_names_language(df_playlists_info[0:10])
    
print(f"Se ha obtenido la información del idioma para {len(language_data)} títulos")

Se ha obtenido la información del idioma para 2659470 títulos


<br>

A continuación, vamos a crear un DataFrame llamado ***df_language*** con el que realizaremos el filtrado de playlists mediante el idioma del título. Este DataFrame contendrá las siguientes columnas:

- ***id***: Identificador de la playlist.
- ***name***: Título de la playlist.
- ***iso6391Name***: Nombre del idioma identificado en formato [_ISO 639-1_](https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes).
- ***score***: Puntuación obtenida por la identificación.

In [27]:
language_dict = dict()

for item in language_data:
    language_dict[item['id']] = item['detectedLanguages'][0]

df_language = pd.DataFrame.from_dict(language_dict, orient='index')

df_language.drop(columns=['name'],inplace=True)

df_language = pd.concat([df_playlists_info['name'], df_language],axis=1, sort=False)
df_language.index.name = 'id'

In [28]:
df_language.head()

,name,iso6391Name,score
id,,,
3FI0aZZz9hUw5qK0K2GDwH,Love Letter,en,1.0
381M0rlWt8mB0CayapCcRr,skin deep,en,1.0
3nSWAjR9QuXhb3YqO24HHw,junior year,en,1.0
5M6FDo7oQ33CvNMaMiZ1uH,BOOM BOOM,en,1.0
43QPMRPsHlbXILXIhYoxF6,Born in the Wrong Era,en,1.0


<br>

Como tenemos títulos que sólo contienen emoticonos, los cuales han sido identificados como '_(Unknown)_', vamos a identificarlos con la etiqueta '_(Emoji)_' y a darles una puntuación de _1_:

In [29]:
df_language.loc[(df_language['iso6391Name'] == '(Unknown)') & 
                (df_language.name.apply(all_text_emoji)), ['iso6391Name']] = '(Emoji)'
df_language.loc[df_language['iso6391Name'] == '(Emoji)',['score']] = 1

<br>

Nuestro principal objetivo es quedaros con aquellos títulos que estén en _inglés_, pero también vamos a seleccionar aquellos títulos cuya etiqueta de idioma sea _('Emoji')_ y _('Unknown')_:

In [30]:
df_language.drop(df_language[(df_language['iso6391Name'] != 'en') &
                             (df_language['iso6391Name'] != '(Emoji)') &
                             (df_language['iso6391Name'] != '(Unknown)')].index,
                 inplace=True)

<br>

También vamos a eliminar aquellos títulos para los cuales su _score es inferior a 0.75_. Esta medida sólo la aplicaremos para los títulos en los que el idioma haya sido identificado como '_en_' (inglés):

In [31]:
df_language.drop(df_language[(df_language['iso6391Name'] == 'en') &
                             (df_language['score'] < 0.75)].index,
                 inplace=True)

<br>

Una vez hemos seleccionado aquellas playlist que nos son de interés, obtenemos sus _id_ y en el DataFrame `df_playlists_info` descartamos aquellas que no se encuentren en el DataFrame `df_language`:

In [32]:
df_playlists_info = df_playlists_info[df_playlists_info.index.isin(df_language.index)]
print(f'Tamaño df_playlists_info (con filtrado de idioma): {len(df_playlists_info)}')

Tamaño df_playlists_info (con filtrado de idioma): 2485960


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section446"></a>
#### <font color="#B20033">Títulos ofensivos</font>

<br>

En este último filtrado, vamos a eliminar los títulos que resulten _ofensivos_. Consideraremos que los títulos ofensivos son aquellos que contienen una o varias palabras como insultos, palabras que pertenezcan a temas considerados _sensibles_, etc...

Tras obtener un diccionario de palabras ofensivas de [_Free Web Headers_](https://www.freewebheaders.com/full-list-of-bad-words-banned-by-google/), vamos a eliminar aquellas playlists que contengan alguna de estas palabras:

In [33]:
bad_words = []

with open('files/bad-words.txt') as f:
    bad_words = f.read().splitlines()

# Palabra ofensiva (única)
single_bad_words = [word for word in bad_words if len(word.split()) == 1]
# Palabras ofensivas (2 o más)
compound_bad_words = [word for word in bad_words if len(word.split()) > 1]

In [34]:
def is_single_bad_name(name):
    return len([word for word in name.split(' ') if word in single_bad_words]) > 0

def is_compound_bad_name(name):
    is_bad = False
    for word in compound_bad_words:
        if word in name:
            is_bad = True
            break
    return is_bad

def is_bad_name(name):
    name = str(name).lower() 
    return is_single_bad_name(name) or is_compound_bad_name(name)

In [35]:
tqdm.pandas()

df_playlists_info.drop(df_playlists_info[df_playlists_info['name'].progress_apply(is_bad_name)].index, inplace=True)
print(f'Tamaño df_playlists (con filtrado de títulos ofensivos): {len(df_playlists_info)}')

100%|█████████████████████████████████████████████████████████████████████| 2485960/2485960 [02:19<00:00, 17819.60it/s]


Tamaño df_playlists (con filtrado de títulos ofensivos): 2392399


<br>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__:
Como el proceso de detección de títulos ofensivos puede tardar varios minutos, hemos añadido una barra de progreso. La documentación de la barra de progreso de _tqdm_ para _pandas_ podemos encontrarla en el siguiente enlace:
- [tqdm: Pandas Integration](https://pypi.org/project/tqdm/#pandas-integration).
</div>


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section45"></a>
### <font color="#B20033">4.5 - Consideraciones en el filtrado de playlists</font>
<br>

Tras realizar el filtrado de playlists, con los criterios que habíamos establecido, no hemos conseguido reducir lo suficiente el número de listas, vamos a aplicar los nuevos criterios que se hemos indicado previamente.

<br>

Como la variable *num_followers* de nuestro conjunto podría ser considerada como una de las más importantes, aumentamos el mínimo de seguidores que tiene una playlist de 1 a 2:

In [36]:
df_playlists_info.drop(df_playlists_info[df_playlists_info['num_followers'] < 2].index,inplace=True)
print(f'Tamaño df_playlists_info (num_followers > 1): {len(df_playlists_info)}')

Tamaño df_playlists_info (num_followers > 1): 1274514


Puesto que todavía no hemos conseguido que el número de playlists esté próximo a 1.000.000, vamos a eliminar aquellas listas donde aparezcan artistas poco frecuentes. Es decir, vamos a comprobar el número de veces que aparece un determinado artista en el conjunto de playlists y si es igual a 1, eliminamos la playlists que lo contiene.

Este proceso se hará de forma recursiva, ya que cuando borremos aquellas playlists con artistas que aparecen una única vez, puede que volvamos a tener listas con éste mismo criterio.

In [37]:
# Cargamos el archivo que contiene la información de los artistas que contiene una playlists
df_pls_artists = pd.read_csv(os.path.join(DATA_PATH,'downloaded_pls_artists.csv'), sep=";", header=None)
df_pls_artists.columns = ['id', 'artists']
df_pls_artists.set_index('id', inplace=True)

In [38]:
# Eliminamos aquellas playlist que se han descartado durante el proceso de filtrado
df_pls_artists = df_pls_artists.loc[df_playlists_info.index]

<br>

Creamos un diccionario en el cual almacenaremos como clave el identificador del artista y cuyo valor será una lista de playlist donde aparece dicho artista:

In [39]:
artists_pls_dict = defaultdict(list)

pbar = tqdm_nb(total=len(df_pls_artists))
for r_id , r_data in df_pls_artists.iterrows():
    pl_id = r_id
    artists = r_data['artists'].split('|')
    for artist in artists:
        artists_pls_dict[artist].append(pl_id)
    pbar.update(1)
pbar.close()

In [40]:
print(f'Número de artistas en el conjunto de datos: {len(artists_pls_dict)}')

Número de artistas en el conjunto de datos: 763499


In [41]:
# Proceso de filtrado de playlists con artistas poco frecuentes
end = False

while not end:
    artists_set = set()
    removable_artists = []
    
    for k,v in artists_pls_dict.items():
        if len(v) <= 1:
            artists_set.update(v)
            removable_artists.append(k)     
            
    if len(removable_artists) == 0 and len(artists_set) == 0:
        end = True
    else:
        for artist in removable_artists:
            del artists_pls_dict[artist]
        for k,v in artists_pls_dict.items():
            artists_pls_dict[k] = [x for x in v if x not in artists_set]
            
print(f'Nuevo número de artistas en el conjunto de datos: {len(artists_pls_dict)}')

Nuevo número de artistas en el conjunto de datos: 237560


Por último, extraemos los identificadores de las playlists del diccionario y filtramos *df_playlists_info* con las listas que hemos descartado:

In [42]:
playlists_set = set()

for _ ,v in artists_pls_dict.items():
    playlists_set.update(v)
    
df_playlists_info = df_playlists_info.loc[list(playlists_set)]
print(f'Tamaño df_playlists_info (con filtrado de artistas): {len(df_playlists_info)}')

Tamaño df_playlists_info (con filtrado de artistas): 1019338


Tras aplicar estos últimos filtros, nos hemos quedado con un número de playlists muy próximo a 1.000.000.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section5"></a>
## <font color="#92002A">5 - Almacenamiento de resultados</font>
<br>

Lo primero que vamos a hacer es aleatorizar las filas del DataFrame `df_playlists_info`, estableciendo la semilla a _1_.

<br>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__:
En el proceso de generación del conjunto de datos, utilizaremos las playlists adicionales que hemos obtenido para crear el conjunto de prueba.
</div> <br>


In [43]:
df_playlists_info = df_playlists_info.sample(n=len(df_playlists_info), random_state=1)

<br>

Si mostramos las primeras 5 columnas que contiene el DataFrame, vemos que las columnas *num_followers*, *num_edits* y *modified_at* son de tipo `float`:


In [44]:
df_playlists_info.head()

,collaborative,name,num_tracks,num_followers,num_artists,num_albums,duration_ms,modified_at,num_edits,file_name
id,,,,,,,,,,
7u1RhzyK1ykPQ1wfemcqoR,False,Low viscosity vibes,58,3.0,25,38,15191903,1.559722e+09,14.0,pls-set_07672.zip
0ZcZqlDEw4eLILfHdni8vG,False,dalanda 🐉,104,4.0,92,100,22361466,1.558851e+09,75.0,pls-set_05170.zip
0Z48DgMwRPZVVTF7Y8RLB6,False,freeze pops,52,2.0,13,25,10643802,1.552264e+09,7.0,pls-set_06675.zip
1G8V8nWYp6uVacS8XAPSbo,False,Golden Oldies,98,2.0,64,90,25418594,1.556949e+09,34.0,pls-set_01145.zip
1mVzj3AosDkLdJOQEtfwaA,False,I d◻n't g◻v◻ a f◻ck,224,2.0,89,154,46528637,1.558704e+09,82.0,pls-set_06752.zip


Como los valores de dichas variables siempre son números enteros, vamos a cambiarlas su tipo a `int64`. También vamos a establecer la variable *collaborative* a tipo `bool`:

In [45]:
df_playlists_info.num_followers = df_playlists_info.num_followers.astype('int64')
df_playlists_info.num_edits = df_playlists_info.num_edits.astype('int64')
df_playlists_info.modified_at = df_playlists_info.modified_at.astype('int64')
df_playlists_info.collaborative = df_playlists_info.collaborative.astype('bool')

df_playlists_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1019338 entries, 7u1RhzyK1ykPQ1wfemcqoR to 03aYLTK68a6Dy6twBW94tY
Data columns (total 10 columns):
collaborative    1019338 non-null bool
name             1019338 non-null object
num_tracks       1019338 non-null int64
num_followers    1019338 non-null int64
num_artists      1019338 non-null int64
num_albums       1019338 non-null int64
duration_ms      1019338 non-null int64
modified_at      1019338 non-null int64
num_edits        1019338 non-null int64
file_name        1019338 non-null object
dtypes: bool(1), int64(7), object(2)
memory usage: 78.7+ MB


Tras aleatorizar las filas del *DataFrame* y corregir el tipo de datos de las columnas, procedemos a almacenar el conjunto con la información de las playlists que vamos a emplear:

In [46]:
file_path = os.path.join(DATA_PATH,'mpd_info_set.csv')       
df_playlists_info.to_csv(file_path, sep=';', encoding='utf-8')

# Guardamos una copia de seguridad
copyfile(file_path, os.path.join(BACKUP_PATH, 'mpd_info_set.csv'))

'backup\\mpd_info_set.csv'

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>